In [1]:
import pandas as pd
import polars as pl
from data import *
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import pickle
import sklearn
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
from scipy import stats
from typing import cast
from utils import *
import wrds

In [17]:
full_data = pl.read_csv(get_data_file_path("signed_predictors_all_wide.csv"), dtypes=ALL_DATA_ENTRY_SCHEMA)

In [18]:
full_data

permno,yyyymm,AM,AOP,AbnormalAccruals,Accruals,AccrualsBM,Activism1,Activism2,AdExp,AgeIPO,AnalystRevision,AnalystValue,AnnouncementReturn,AssetGrowth,BM,BMdec,BPEBM,Beta,BetaFP,BetaLiquidityPS,BetaTailRisk,BidAskSpread,BookLeverage,BrandInvest,CBOperProf,CF,CPVolSpread,Cash,CashProd,ChAssetTurnover,ChEQ,ChForecastAccrual,ChInv,ChInvIA,ChNAnalyst,ChNNCOA,…,Spinoff,SurpriseRD,Tax,TotalAccruals,TrendFactor,UpRecomm,VarCF,VolMkt,VolSD,VolumeTrend,XFIN,betaVIX,cfp,dCPVolSpread,dNoa,dVolCall,dVolPut,fgr5yrLag,grcapx,grcapx3y,hire,iomom_cust,iomom_supp,realestate,retConglomerate,roaq,sfe,sinAlgo,skew1,std_turn,tang,zerotrade,zerotradeAlt1,zerotradeAlt12,STreversal,Price,Size
i64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
10000,"""198601""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,0.0,null,null,null,null,null,null,null,null,null,null,-0.005234,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,0.0,-1.475907,-9.686575
10000,"""198602""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,0.0,null,null,null,null,null,null,null,null,null,null,-0.003488,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,4.7852e-8,null,25.7143,-1.178655,-9.389323
10000,"""198603""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,0.0,null,null,null,null,null,null,null,null,null,null,-0.002715,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1.0234e-7,null,-36.5385,-1.490091,-9.700759
10000,"""198604""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,0.0,null,null,null,null,null,null,null,null,null,null,0.000877,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,7.4675e-8,null,9.8592,-1.386294,-9.627207
10000,"""198605""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,0.0,null,null,null,null,null,null,null,null,null,null,0.008818,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,7.6496e-8,null,22.2656,-1.134423,-9.375336
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
93433,"""201005""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,0.0,null,null
93434,"""201005""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,0.0,null,null
93434,"""202312""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,n

In [5]:
db = wrds.Connection(wrds_username=get_env("WRDS_USERNAME"))

# Query to get the stock returns
query = """
SELECT permno, date, ret
FROM crsp.msf
WHERE date BETWEEN '1986-01-01' AND '2024-05-31'
"""

# Execute the query
all_returns_data = db.raw_sql(query)

# Close the connection
db.close()

Loading library list...
Done


In [6]:
all_returns_data

,permno,date,ret
0,10000,1986-01-31,NaN
1,10000,1986-02-28,-0.257143
2,10000,1986-03-31,0.365385
3,10000,1986-04-30,-0.098592
4,10000,1986-05-30,-0.222656
...,...,...,...
487268,93436,2023-08-31,-0.034962
487269,93436,2023-09-29,-0.030456
487270,93436,2023-10-31,-0.197346
487271,93436,2023-11-30,0.195379


In [8]:
all_returns_data.to_csv(get_data_file_path("all_returns_values.csv")) # type: ignore

In [9]:
all_returns_df = pl.read_csv(get_data_file_path("all_returns_values.csv"), dtypes={"permno": pl.Int64, "date": pl.Date, "ret": pl.Float64})

In [10]:
all_returns_df

,permno,date,ret
i64,i64,date,f64
0,10000,1986-01-31,null
1,10000,1986-02-28,-0.257143
2,10000,1986-03-31,0.365385
3,10000,1986-04-30,-0.098592
4,10000,1986-05-30,-0.222656
…,…,…,…
487268,93436,2023-08-31,-0.034962
487269,93436,2023-09-29,-0.030456
487270,93436,2023-10-31,-0.197346


In [11]:
all_returns_df.insert_column(2, pl.Series("yyyymm", all_returns_df.get_column("date").apply(lambda x: x.strftime("%Y%m"))))

/var/folders/qk/3qb7y7bs23l46_jb_xbrk_zw0000gn/T/ipykernel_99231/1299984043.py:1: DeprecationWarning: `apply` is deprecated. It has been renamed to `map_elements`.
  all_returns_df.insert_column(2, pl.Series("yyyymm", all_returns_df.get_column("date").apply(lambda x: x.strftime("%Y%m"))))
/var/folders/qk/3qb7y7bs23l46_jb_xbrk_zw0000gn/T/ipykernel_99231/1299984043.py:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  all_returns_df.insert_column(2, pl.Series("yyyymm", all_returns_df.get_column("date").apply(lambda x: x.strftime("%Y%m"))))


,permno,yyyymm,date,ret
i64,i64,str,date,f64
0,10000,"""198601""",1986-01-31,null
1,10000,"""198602""",1986-02-28,-0.257143
2,10000,"""198603""",1986-03-31,0.365385
3,10000,"""198604""",1986-04-30,-0.098592
4,10000,"""198605""",1986-05-30,-0.222656
…,…,…,…,…
487268,93436,"""202308""",2023-08-31,-0.034962
487269,93436,"""202309""",2023-09-29,-0.030456
487270,93436,"""202310""",2023-10-31,-0.197346


In [15]:
all_returns_df.drop_in_place("")

""
i64
0
1
2
3
4
…
487268
487269
487270


In [19]:
all_returns_df

permno,yyyymm,date,ret
i64,str,date,f64
10000,"""198601""",1986-01-31,null
10000,"""198602""",1986-02-28,-0.257143
10000,"""198603""",1986-03-31,0.365385
10000,"""198604""",1986-04-30,-0.098592
10000,"""198605""",1986-05-30,-0.222656
…,…,…,…
93436,"""202308""",2023-08-31,-0.034962
93436,"""202309""",2023-09-29,-0.030456
93436,"""202310""",2023-10-31,-0.197346


In [20]:
full_data = full_data.join(all_returns_df, on=("permno", "yyyymm"))

In [23]:
full_data.write_csv(get_data_file_path("COMPLETE_DATASET.csv"))

In [3]:
data = pl.read_csv(get_data_file_path("signed_predictors_dl_wide.csv"), dtypes=SIGNED_PREDICTORS_ENTRY_SCHEMA)
# data = pd.read_csv(get_data_file_path("signed_predictors_dl_wide.csv"), dtype=SIGNED_PREDICTORS_ENTRY_SCHEMA)

In [4]:
data

permno,yyyymm,AM,AOP,AbnormalAccruals,Accruals,AccrualsBM,Activism1,Activism2,AdExp,AgeIPO,AnalystRevision,AnalystValue,AnnouncementReturn,AssetGrowth,BM,BMdec,BPEBM,Beta,BetaFP,BetaLiquidityPS,BetaTailRisk,BidAskSpread,BookLeverage,BrandInvest,CBOperProf,CF,CPVolSpread,Cash,CashProd,ChAssetTurnover,ChEQ,ChForecastAccrual,ChInv,ChInvIA,ChNAnalyst,ChNNCOA,…,ShareVol,ShortInterest,SmileSlope,Spinoff,SurpriseRD,Tax,TotalAccruals,TrendFactor,UpRecomm,VarCF,VolMkt,VolSD,VolumeTrend,XFIN,betaVIX,cfp,dCPVolSpread,dNoa,dVolCall,dVolPut,fgr5yrLag,grcapx,grcapx3y,hire,iomom_cust,iomom_supp,realestate,retConglomerate,roaq,sfe,sinAlgo,skew1,std_turn,tang,zerotrade,zerotradeAlt1,zerotradeAlt12
i64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
10000,"""198601""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,-1.0,null,null,0.0,null,null,null,null,null,null,null,null,null,null,-0.005234,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
10000,"""198602""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,-1.0,null,null,0.0,null,null,null,null,null,null,null,null,null,null,-0.003488,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,4.7852e-8,null
10000,"""198603""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,0.0,null,null,0.0,null,null,null,null,null,null,null,null,null,null,-0.002715,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1.0234e-7,null
10000,"""198604""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,0.0,null,null,null,null,null,null,null,null,null,null,0.000877,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,7.4675e-8,null
10000,"""198605""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,0.0,null,null,null,null,null,null,null,null,null,null,0.008818,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,7.6496e-8,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
93436,"""202401""",null,null,-0.003877,0.016862,null,null,null,null,null,null,null,null,-0.325232,null,null,null,null,null,null,null,null,-1.838476,null,null,null,null,null,null,0.598015,-1.480804,null,-0.098042,null,null,0.097266,…,null,null,null,null,0.0,0.288081,-0.223286,null,null,null,null,null,null,0.040382,null,null,null,-0.109511,null,null,null,null,null,-0.251513,null,null,null,null,null,null,null,null,null,0.698178,null,null,null
93436,"""202402""",null,null,-0.003877,0.016862,null,null,null,null,null,null,null,null,-0.325232,null,null,null,null,null,null,null,null,-1.838476,null,null,null,null,null,null,0.598015,-1.480804,null,-0.098042,null,null,0.097266,…,null,null,null,null,0.0,0.288081,-0.223286,null,null,null,null,null,null,0.040382,null,null,null,-0.109511,null,null,null,null,null,-0.251513,null,null,null,null,null,null,null,null,null,0.698178,null,null,null
93436,"""202403""",null,null,-0.

In [18]:
# Handle missing values
imputer = SimpleImputer(strategy="mean")
all_predictors_imputed = imputer.fit_transform(data)

In [21]:
with open(get_data_file_path("all_predictors_imputed.pkl"), "wb") as f:
    pickle.dump(all_predictors_imputed, f)

In [3]:
with open(get_data_file_path("all_predictors_imputed.pkl"), "rb") as f:
    all_predictors_imputed = pickle.load(f)

In [5]:
data_imputed = pl.DataFrame(all_predictors_imputed, schema=SIGNED_PREDICTORS_ENTRY_SCHEMA)

In [6]:
_ = data_imputed.replace("yyyymm", data_imputed.get_column("yyyymm").str.strip_chars(".0"))
del _

/var/folders/qk/3qb7y7bs23l46_jb_xbrk_zw0000gn/T/ipykernel_19284/83776391.py:1: DeprecationWarning: `replace` is deprecated. DataFrame.replace is deprecated and will be removed in a future version. Please use
    df = df.with_columns(new_column.alias(column_name))
instead.
  _ = data_imputed.replace("yyyymm", data_imputed.get_column("yyyymm").str.strip_chars(".0"))


In [8]:
with open(get_data_file_path("all_predictors_imputed_df.pkl"), "wb") as f:
    pickle.dump(data_imputed, f)

In [3]:
with open(get_data_file_path("all_predictors_imputed_df.pkl"), "rb") as f:
    data_imputed: pl.DataFrame = pickle.load(f)

In [4]:
data_imputed

permno,yyyymm,AM,AOP,AbnormalAccruals,Accruals,AccrualsBM,Activism1,Activism2,AdExp,AgeIPO,AnalystRevision,AnalystValue,AnnouncementReturn,AssetGrowth,BM,BMdec,BPEBM,Beta,BetaFP,BetaLiquidityPS,BetaTailRisk,BidAskSpread,BookLeverage,BrandInvest,CBOperProf,CF,CPVolSpread,Cash,CashProd,ChAssetTurnover,ChEQ,ChForecastAccrual,ChInv,ChInvIA,ChNAnalyst,ChNNCOA,…,ShareVol,ShortInterest,SmileSlope,Spinoff,SurpriseRD,Tax,TotalAccruals,TrendFactor,UpRecomm,VarCF,VolMkt,VolSD,VolumeTrend,XFIN,betaVIX,cfp,dCPVolSpread,dNoa,dVolCall,dVolPut,fgr5yrLag,grcapx,grcapx3y,hire,iomom_cust,iomom_supp,realestate,retConglomerate,roaq,sfe,sinAlgo,skew1,std_turn,tang,zerotrade,zerotradeAlt1,zerotradeAlt12
i64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
10000,"""198601""",3.679108,-165.572065,0.000082,0.031046,0.483201,14.8865,9.263089,0.075071,18.541335,1.006447,0.756109,0.00265,-0.176024,-0.726098,2.955261,0.010867,0.989357,0.973176,-0.000942,0.641241,0.016827,-4.025639,-2575.475564,0.093091,0.009326,-0.009441,0.163636,12.959522,-0.05317,-1.314422,0.4765,-0.010126,1184.389561,-0.145632,0.003044,…,-1.0,-32793.262891,-0.005706,0.0,0.287186,1.166194,-0.003901,0.215255,0.352181,-0.86924,-0.170444,-4.679102,-0.005833,-0.062235,-0.005234,0.004548,0.000138,-0.098494,0.000396,-0.000534,-16.780281,-3.378635,-3.0530e10,-0.036986,1.725571,1.59892,-5.7231e-11,0.010997,-0.007297,-0.002534,0.17759,-0.067404,-0.168298,0.692479,1.483897,1.569696,1.417246
10000,"""198602""",3.679108,-165.572065,0.000082,0.031046,0.483201,14.8865,9.263089,0.075071,18.541335,1.006447,0.756109,0.00265,-0.176024,-0.726098,2.955261,0.010867,0.989357,0.973176,-0.000942,0.641241,0.016827,-4.025639,-2575.475564,0.093091,0.009326,-0.009441,0.163636,12.959522,-0.05317,-1.314422,0.4765,-0.010126,1184.389561,-0.145632,0.003044,…,-1.0,-32793.262891,-0.005706,0.0,0.287186,1.166194,-0.003901,0.215255,0.352181,-0.86924,-0.170444,-4.679102,-0.005833,-0.062235,-0.003488,0.004548,0.000138,-0.098494,0.000396,-0.000534,-16.780281,-3.378635,-3.0530e10,-0.036986,1.725571,1.59892,-5.7231e-11,0.010997,-0.007297,-0.002534,0.17759,-0.067404,-0.168298,0.692479,1.483897,4.7852e-8,1.417246
10000,"""198603""",3.679108,-165.572065,0.000082,0.031046,0.483201,14.8865,9.263089,0.075071,18.541335,1.006447,0.756109,0.00265,-0.176024,-0.726098,2.955261,0.010867,0.989357,0.973176,-0.000942,0.641241,0.016827,-4.025639,-2575.475564,0.093091,0.009326,-0.009441,0.163636,12.959522,-0.05317,-1.314422,0.4765,-0.010126,1184.389561,-0.145632,0.003044,…,0.0,-32793.262891,-0.005706,0.0,0.287186,1.166194,-0.003901,0.215255,0.352181,-0.86924,-0.170444,-4.679102,-0.005833,-0.062235,-0.002715,0.004548,0.000138,-0.098494,0.000396,-0.000534,-16.780281,-3.378635,-3.0530e10,-0.036986,1.725571,1.59892,-5.7231e-11,0.010997,-0.007297,-0.002534,0.17759,-0.067404,-0.168298,0.692479,1.483897,1.0234e-7,1.417246
10000,"""198604""",3.679108,-165.572065,0.000082,0.031046,0.483201,14.8865,9.263089,0.075071,18.541335,1.006447,0.756109,0.00265,-0.176024,-0.726098,2.955261,0.010867,0.989357,0.973176,-0.000942,0.641241,0.016827,-4.025639,-2575.475564,0.093091,0.009326,-0.009441,0.163636,12.959522,-0.05317,-1.314422,0.4765,-0.010126,1184.389561,-0.145632,0.003044,…,-0.30328,-32793.262891,-0.005706,0.0,0.287186,1.166194,-0.003901,0.215255,0.352181,-0.86924,-0.170444,-4.679102,-0.005833,-0.062235,0.000877,0.004548,0.000138,-0.098494,0.000396,-0.000534,-16.780281,-3.378635,-3.0530e10,-0.036986,1.725571,1.59892,-5.7231e-11,0.010997,-0.007297,-0.002534,0.17759,-0.067404,-0.168298,0.692479,1.483897,7.4675e-8,1.417246
10000,"""198605""",3.679108,-165.572065,0.000082,0.031046,0.483201,14.8865,9.263089,0.075071,18.541335,1.006447,0.756109,0.00265,-0.176024,-0.726098,2.955261,0.010867,0.989357,0.973176,-0.000942,0.641241

In [6]:
ret = pl.read_csv(get_data_file_path("PredictorPortsFull.csv"), null_values=("NA",), dtypes=PREDICTOR_PORTS_ENTRY_SCHEMA)

In [7]:
ret

signalname,port,date,ret,signallag,Nlong,Nshort
str,str,date,f64,f64,i64,i64
"""AM""","""01""",1951-07-31,7.661648,0.681619,67,0
"""AM""","""01""",1951-08-31,4.273648,0.639816,67,0
"""AM""","""01""",1951-09-28,1.31527,0.617079,67,0
"""AM""","""01""",1951-10-31,-3.94298,0.612266,67,0
"""AM""","""01""",1951-11-30,1.02867,0.63771,67,0
…,…,…,…,…,…,…
"""zerotradeAlt12""","""LS""",2022-08-31,-1.467892,null,491,492
"""zerotradeAlt12""","""LS""",2022-09-30,4.607782,null,490,491
"""zerotradeAlt12""","""LS""",2022-10-31,-5.81855,null,492,492


In [8]:
ret.insert_column(3, pl.Series("yyyymm", ret.get_column("date").apply(lambda x: x.strftime("%Y%m"))))

/var/folders/qk/3qb7y7bs23l46_jb_xbrk_zw0000gn/T/ipykernel_89187/3833254137.py:1: DeprecationWarning: `apply` is deprecated. It has been renamed to `map_elements`.
  ret.insert_column(3, pl.Series("yyyymm", ret.get_column("date").apply(lambda x: x.strftime("%Y%m"))))
/var/folders/qk/3qb7y7bs23l46_jb_xbrk_zw0000gn/T/ipykernel_89187/3833254137.py:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  ret.insert_column(3, pl.Series("yyyymm", ret.get_column("date").apply(lambda x: x.strftime("%Y%m"))))


signalname,port,date,yyyymm,ret,signallag,Nlong,Nshort
str,str,date,str,f64,f64,i64,i64
"""AM""","""01""",1951-07-31,"""195107""",7.661648,0.681619,67,0
"""AM""","""01""",1951-08-31,"""195108""",4.273648,0.639816,67,0
"""AM""","""01""",1951-09-28,"""195109""",1.31527,0.617079,67,0
"""AM""","""01""",1951-10-31,"""195110""",-3.94298,0.612266,67,0
"""AM""","""01""",1951-11-30,"""195111""",1.02867,0.63771,67,0
…,…,…,…,…,…,…,…
"""zerotradeAlt12""","""LS""",2022-08-31,"""202208""",-1.467892,null,491,492
"""zerotradeAlt12""","""LS""",2022-09-30,"""202209""",4.607782,null,490,491
"""zerotradeAlt12""","""LS""",2022-10-31,"""202210""",-5.81855,null,492,492


In [9]:
ret

signalname,port,date,yyyymm,ret,signallag,Nlong,Nshort
str,str,date,str,f64,f64,i64,i64
"""AM""","""01""",1951-07-31,"""195107""",7.661648,0.681619,67,0
"""AM""","""01""",1951-08-31,"""195108""",4.273648,0.639816,67,0
"""AM""","""01""",1951-09-28,"""195109""",1.31527,0.617079,67,0
"""AM""","""01""",1951-10-31,"""195110""",-3.94298,0.612266,67,0
"""AM""","""01""",1951-11-30,"""195111""",1.02867,0.63771,67,0
…,…,…,…,…,…,…,…
"""zerotradeAlt12""","""LS""",2022-08-31,"""202208""",-1.467892,null,491,492
"""zerotradeAlt12""","""LS""",2022-09-30,"""202209""",4.607782,null,490,491
"""zerotradeAlt12""","""LS""",2022-10-31,"""202210""",-5.81855,null,492,492


# MAIN START

In [2]:
full_data = pl.read_csv(get_data_file_path("COMPLETE_DATASET.csv"))

In [3]:
full_data

permno,yyyymm,AM,AOP,AbnormalAccruals,Accruals,AccrualsBM,Activism1,Activism2,AdExp,AgeIPO,AnalystRevision,AnalystValue,AnnouncementReturn,AssetGrowth,BM,BMdec,BPEBM,Beta,BetaFP,BetaLiquidityPS,BetaTailRisk,BidAskSpread,BookLeverage,BrandInvest,CBOperProf,CF,CPVolSpread,Cash,CashProd,ChAssetTurnover,ChEQ,ChForecastAccrual,ChInv,ChInvIA,ChNAnalyst,ChNNCOA,…,Tax,TotalAccruals,TrendFactor,UpRecomm,VarCF,VolMkt,VolSD,VolumeTrend,XFIN,betaVIX,cfp,dCPVolSpread,dNoa,dVolCall,dVolPut,fgr5yrLag,grcapx,grcapx3y,hire,iomom_cust,iomom_supp,realestate,retConglomerate,roaq,sfe,sinAlgo,skew1,std_turn,tang,zerotrade,zerotradeAlt1,zerotradeAlt12,STreversal,Price,Size,date,ret
i64,i64,f64,str,f64,f64,str,str,str,f64,str,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,f64,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,str,str,f64,f64,f64,f64,f64,f64,f64,str,f64,str,str,str,f64,f64,f64,f64,f64,str,str,f64,str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,str,f64
10000,198601,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,-0.005234,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,0.0,-1.475907,-9.686575,"""1986-01-31""",null
10000,198602,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,-0.003488,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,4.7852e-8,null,25.7143,-1.178655,-9.389323,"""1986-02-28""",-0.257143
10000,198603,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,-0.002715,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1.0234e-7,null,-36.5385,-1.490091,-9.700759,"""1986-03-31""",0.365385
10000,198604,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,0.000877,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,7.4675e-8,null,9.8592,-1.386294,-9.627207,"""1986-04-30""",-0.098592
10000,198605,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,0.008818,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,7.6496e-8,null,22.2656,-1.134423,-9.375336,"""1986-05-30""",-0.222656
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
93433,201005,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,0.0,null,null,"""2010-05-28""",null
93434,201005,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,0.0,null,null,"""2010-05-28""",null
93434,202312,null,null,null,null,null,null,null,null,n

In [4]:
# Get number of non-null elements
full_data.count() / len(full_data)

permno,yyyymm,AM,AOP,AbnormalAccruals,Accruals,AccrualsBM,Activism1,Activism2,AdExp,AgeIPO,AnalystRevision,AnalystValue,AnnouncementReturn,AssetGrowth,BM,BMdec,BPEBM,Beta,BetaFP,BetaLiquidityPS,BetaTailRisk,BidAskSpread,BookLeverage,BrandInvest,CBOperProf,CF,CPVolSpread,Cash,CashProd,ChAssetTurnover,ChEQ,ChForecastAccrual,ChInv,ChInvIA,ChNAnalyst,ChNNCOA,…,Tax,TotalAccruals,TrendFactor,UpRecomm,VarCF,VolMkt,VolSD,VolumeTrend,XFIN,betaVIX,cfp,dCPVolSpread,dNoa,dVolCall,dVolPut,fgr5yrLag,grcapx,grcapx3y,hire,iomom_cust,iomom_supp,realestate,retConglomerate,roaq,sfe,sinAlgo,skew1,std_turn,tang,zerotrade,zerotradeAlt1,zerotradeAlt12,STreversal,Price,Size,date,ret
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
1.0,1.0,0.606123,0.299546,0.551415,0.637375,0.044407,0.031179,0.008651,0.216638,0.0839,0.461049,0.299546,0.674647,0.640398,0.53409,0.617441,0.602105,0.808998,0.717246,0.716446,0.403281,0.906246,0.70211,0.095517,0.449682,0.606123,0.190793,0.489377,0.601554,0.473897,0.611098,0.14819,0.64041,0.536361,0.044802,0.635248,…,0.622181,0.605137,0.336718,0.12718,0.513814,0.874562,0.789564,0.735561,0.62337,0.900945,0.566054,0.228818,0.639837,0.228818,0.228818,0.225556,0.489308,0.449548,0.691628,0.449334,0.449607,0.300931,0.185339,0.575195,0.143186,0.03551,0.131394,0.448143,0.271994,0.904986,0.937796,0.853177,1.0,0.973384,0.973384,1.0,0.965786


In [5]:
full_data

permno,yyyymm,AM,AOP,AbnormalAccruals,Accruals,AccrualsBM,Activism1,Activism2,AdExp,AgeIPO,AnalystRevision,AnalystValue,AnnouncementReturn,AssetGrowth,BM,BMdec,BPEBM,Beta,BetaFP,BetaLiquidityPS,BetaTailRisk,BidAskSpread,BookLeverage,BrandInvest,CBOperProf,CF,CPVolSpread,Cash,CashProd,ChAssetTurnover,ChEQ,ChForecastAccrual,ChInv,ChInvIA,ChNAnalyst,ChNNCOA,…,Tax,TotalAccruals,TrendFactor,UpRecomm,VarCF,VolMkt,VolSD,VolumeTrend,XFIN,betaVIX,cfp,dCPVolSpread,dNoa,dVolCall,dVolPut,fgr5yrLag,grcapx,grcapx3y,hire,iomom_cust,iomom_supp,realestate,retConglomerate,roaq,sfe,sinAlgo,skew1,std_turn,tang,zerotrade,zerotradeAlt1,zerotradeAlt12,STreversal,Price,Size,date,ret
i64,i64,f64,str,f64,f64,str,str,str,f64,str,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,f64,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,str,str,f64,f64,f64,f64,f64,f64,f64,str,f64,str,str,str,f64,f64,f64,f64,f64,str,str,f64,str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,str,f64
10000,198601,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,-0.005234,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,0.0,-1.475907,-9.686575,"""1986-01-31""",null
10000,198602,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,-0.003488,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,4.7852e-8,null,25.7143,-1.178655,-9.389323,"""1986-02-28""",-0.257143
10000,198603,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,-0.002715,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1.0234e-7,null,-36.5385,-1.490091,-9.700759,"""1986-03-31""",0.365385
10000,198604,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,0.000877,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,7.4675e-8,null,9.8592,-1.386294,-9.627207,"""1986-04-30""",-0.098592
10000,198605,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,0.008818,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,7.6496e-8,null,22.2656,-1.134423,-9.375336,"""1986-05-30""",-0.222656
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
93433,201005,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,0.0,null,null,"""2010-05-28""",null
93434,201005,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,0.0,null,null,"""2010-05-28""",null
93434,202312,null,null,null,null,null,null,null,null,n

In [ ]:
full_data.select

In [ ]:
def run_models_and_plot(df, target_column):
    # Separate features (X) and target (y)
    X = df.drop(target_column, axis=1)
    y = df[target_column]

    # Normalize the features
    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    # Split the data into training and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Create and fit models
    models = [
        LinearRegression(),
        Ridge(),
        Lasso(alpha=0.1),
        RandomForestRegressor(),
        AdaBoostRegressor(),
        XGBRegressor(objective ="reg:squarederror")
    ]
    model_names = ["LinearRegression", "Ridge", "Lasso", "RandomForestRegressor", "AdaBoostRegressor", "XGBRegressor"]

    # Make predictions using each model and plot observed vs fitted values
    for model, name in zip(models, model_names):
        model.fit(X_train, y_train)
        if name == "Lasso":
            print(model.coef_)
        predictions = model.predict(X_test)

        # Plot observed vs fitted values
        plt.figure(figsize=(6, 6))
        plt.scatter(y_test, predictions)
        plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], "k--", lw=4)
        plt.xlabel("Observed")
        plt.ylabel("Fitted")
        plt.title(name)
        plt.show()

run_models_and_plot(listings_1, "price")